# Hello SQL - Jobs

## Import db as world

In [47]:
import pandas as pd
import sqlite3

# Load the CSV file into a DataFrame
df = pd.read_csv('../data/countries_of_the_world.csv')

# Clean column names
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('.', '').str.replace('$', '').str.replace('%', '')

# Rename columns to match job expectations
rename_dict = {
    'country': 'name',
    'region': 'region',
    'population': 'population',
    'area_sq_mi': 'area',
    'pop_density_per_sq_mi': 'density',
    'coastline_coast/area_ratio': 'coastline',
    'net_migration': 'net_migration',
    'infant_mortality_per_1000_births': 'infant_mortality',
    'gdp__per_capita': 'gdp_per_capita',
    'literacy_': 'literacy',
    'phones_per_1000': 'phones',
    'arable_': 'arable',
    'crops_': 'crops',
    'other_': 'other',
    'climate': 'climate',
    'birthrate': 'birthrate',
    'deathrate': 'deathrate',
    'agriculture': 'agriculture',
    'industry': 'industry',
    'service': 'service'
}
df = df.rename(columns=rename_dict)

# Strip spaces from string columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()

# Convert numeric columns (replace ',' with '.' and convert to float)
numeric_cols = ['population', 'area', 'density', 'coastline', 'net_migration', 'infant_mortality', 'gdp_per_capita', 'literacy', 'phones', 'arable', 'crops', 'other', 'birthrate', 'deathrate', 'agriculture', 'industry', 'service']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Insert the DataFrame into the database as a table named 'world'
df.to_sql('world', conn, index=False, if_exists='replace')

227

Check if good import

In [48]:
# Query the 'world' table
result = pd.read_sql('SELECT * FROM world LIMIT 5', conn)
print(result)

             name                region  population     area  density  \
0     Afghanistan  ASIA (EX. NEAR EAST)    31056997   647500     48.0   
1         Albania        EASTERN EUROPE     3581655    28748    124.6   
2         Algeria       NORTHERN AFRICA    32930091  2381740     13.8   
3  American Samoa               OCEANIA       57794      199    290.4   
4         Andorra        WESTERN EUROPE       71201      468    152.1   

   coastline  net_migration  infant_mortality  gdp_per_capita  literacy  \
0       0.00          23.06            163.07           700.0      36.0   
1       1.26          -4.93             21.52          4500.0      86.5   
2       0.04          -0.39             31.00          6000.0      70.0   
3      58.29         -20.71              9.27          8000.0      97.0   
4       0.00           6.60              4.05         19000.0     100.0   

   phones  arable  crops  other climate  birthrate  deathrate  agriculture  \
0     3.2   12.13   0.22  87.65 

## Job 1

In [49]:
print("1")
result1 = pd.read_sql("SELECT name, population FROM world WHERE name = 'Germany';", conn)
print(result1)

print("\n2")
result2 = pd.read_sql("SELECT name, population FROM world WHERE name IN ('Sweden', 'Norway', 'Denmark');", conn)
print(result2)

print("\n3")
result3 = pd.read_sql("SELECT name, area FROM world WHERE area > 200000 AND area < 300000;", conn)
print(result3)

1
      name  population
0  Germany    82422299

2
      name  population
0  Denmark     5450661
1   Norway     4610820
2   Sweden     9016596

3
              name    area
0          Belarus  207600
1     Burkina Faso  274200
2          Ecuador  283560
3            Gabon  267667
4            Ghana  239460
5           Guinea  245857
6           Guyana  214970
7             Laos  236800
8      New Zealand  268680
9             Oman  212460
10         Romania  237500
11          Uganda  236040
12  United Kingdom  244820
13  Western Sahara  266000


## Job 2

In [50]:
print("1")
result = pd.read_sql("SELECT name FROM world WHERE name LIKE 'B%';", conn)
print(result)

print("\n2")
result = pd.read_sql("SELECT name FROM world WHERE name LIKE 'Al%';", conn)
print(result)

print("\n3")
result = pd.read_sql("SELECT name FROM world WHERE name LIKE '%y';", conn)
print(result)

print("\n4")
result = pd.read_sql("SELECT name FROM world WHERE name LIKE '%land';", conn)
print(result)

print("\n5")
result = pd.read_sql("SELECT name FROM world WHERE LOWER(name) LIKE '%w%';", conn)
print(result)

print("\n6")
result = pd.read_sql("SELECT name FROM world WHERE name LIKE '%oo%' OR name LIKE '%ee%';", conn)
print(result)

print("\n7")
result = pd.read_sql("SELECT name FROM world WHERE (LENGTH(LOWER(name)) - LENGTH(REPLACE(LOWER(name), 'a', ''))) >= 3;", conn)
print(result)

print("\n8")
result = pd.read_sql("SELECT name FROM world WHERE SUBSTR(name, 2, 1) = 'r';", conn)
print(result)

1
                    name
0           Bahamas, The
1                Bahrain
2             Bangladesh
3               Barbados
4                Belarus
5                Belgium
6                 Belize
7                  Benin
8                Bermuda
9                 Bhutan
10               Bolivia
11  Bosnia & Herzegovina
12              Botswana
13                Brazil
14    British Virgin Is.
15                Brunei
16              Bulgaria
17          Burkina Faso
18                 Burma
19               Burundi

2
      name
0  Albania
1  Algeria

3
       name
0   Germany
1  Guernsey
2   Hungary
3     Italy
4    Jersey
5    Norway
6  Paraguay
7    Turkey
8   Uruguay

4
          name
0      Finland
1    Greenland
2      Iceland
3      Ireland
4  New Zealand
5       Poland
6    Swaziland
7  Switzerland
8     Thailand

5
                 name
0            Botswana
1              Kuwait
2              Malawi
3       New Caledonia
4         New Zealand
5              Norway
6   

## Job 3

In [51]:
print("\n0 - Create Students Table")
conn.execute("CREATE TABLE IF NOT EXISTS students (student_id INTEGER PRIMARY KEY, first_name CHAR(15), last_name CHAR(15), age INTEGER, grade CHAR(3));")
conn.execute("INSERT INTO students (first_name, last_name, age, grade) VALUES ('Alice', 'Johnson', 22, 'A+'), ('Bob', 'Smith', 20, 'B'), ('Charlie', 'Williams', 21, 'C'), ('David', 'Brown', 23, 'B+'), ('Eva', 'David', 19, 'A'), ('Frank', 'Jones', 22, 'C+');")


print("\n1")
result = pd.read_sql("SELECT * FROM students;", conn)
print(result)

print("\n2")
result = pd.read_sql("SELECT * FROM students WHERE age > 20;", conn)
print(result)

print("\n3")
result = pd.read_sql("SELECT * FROM students ORDER BY grade ASC;", conn)
print(result)

print("\nCustom Grades")
result = pd.read_sql("SELECT *, CASE grade WHEN 'A+' THEN 1 WHEN 'A' THEN 2 WHEN 'B+' THEN 3 WHEN 'B' THEN 4 WHEN 'C+' THEN 5 WHEN 'C' THEN 6 ELSE 99 END AS grade_rank FROM students ORDER BY grade_rank ASC;", conn)
print(result)


0 - Create Students Table

1
   student_id first_name last_name  age grade
0           1      Alice   Johnson   22    A+
1           2        Bob     Smith   20     B
2           3    Charlie  Williams   21     C
3           4      David     Brown   23    B+
4           5        Eva     David   19     A
5           6      Frank     Jones   22    C+

2
   student_id first_name last_name  age grade
0           1      Alice   Johnson   22    A+
1           3    Charlie  Williams   21     C
2           4      David     Brown   23    B+
3           6      Frank     Jones   22    C+

3
   student_id first_name last_name  age grade
0           5        Eva     David   19     A
1           1      Alice   Johnson   22    A+
2           2        Bob     Smith   20     B
3           4      David     Brown   23    B+
4           3    Charlie  Williams   21     C
5           6      Frank     Jones   22    C+

Custom Grades
   student_id first_name last_name  age grade  grade_rank
0           1    

## Job 4

In [52]:
print("\n0 - Create Students Table")
conn.execute("CREATE TABLE IF NOT EXISTS nobel (yr INTEGER, subject CHAR(15), winner CHAR(155));")
conn.execute("INSERT INTO nobel (yr, subject, winner) VALUES (1960, 'Chemistry', 'Willard F.Libby'), (1960, 'Literature', 'Saint_John Perse'), (1960, 'Medicine', 'Sir Frank Macfariane Burnet'), (1960, 'Medicine', 'Peter Madawar');")


print("1")
result = pd.read_sql("SELECT yr, subject, winner FROM nobel WHERE yr = 1986;", conn)
print(result)

print("\n2")
result = pd.read_sql("SELECT yr, subject, winner FROM nobel WHERE yr = 1967 AND subject = 'Literature';", conn)
print(result)

print("\n3")
result = pd.read_sql("SELECT yr, subject FROM nobel WHERE winner LIKE '%Albert Einstein%';", conn)
print(result)

print("\n4")
result = pd.read_sql("SELECT yr, subject, winner FROM nobel WHERE subject = 'Literature' AND yr BETWEEN 1980 AND 1989 ORDER BY yr;", conn)
print(result)

print("\n5")
result = pd.read_sql("SELECT yr, subject, winner FROM nobel WHERE subject = 'Mathematics';", conn)
print(result)
print("\nCount:")
result = pd.read_sql("SELECT COUNT(*) AS math_winners_count FROM nobel WHERE subject = 'Mathematics';", conn)
print(result)


0 - Create Students Table
1
Empty DataFrame
Columns: [yr, subject, winner]
Index: []

2
Empty DataFrame
Columns: [yr, subject, winner]
Index: []

3
Empty DataFrame
Columns: [yr, subject]
Index: []

4
Empty DataFrame
Columns: [yr, subject, winner]
Index: []

5
Empty DataFrame
Columns: [yr, subject, winner]
Index: []

Count:
   math_winners_count
0                   0


## Job 5

In [53]:
print("1")
result = pd.read_sql("SELECT w.name, w.population FROM world w WHERE w.population > (SELECT population FROM world WHERE name = 'Russia');", conn)
print(result)

print("\n2")
result = pd.read_sql("SELECT name, gdp_per_capita FROM world WHERE region in ('WESTERN EUROPE', 'EASTERN EUROPE') AND gdp_per_capita > (SELECT gdp_per_capita FROM world WHERE name = 'Italy');", conn)
print(result)

print("\n3")
result = pd.read_sql("SELECT name, population FROM world WHERE population > (SELECT population FROM world WHERE name = 'United Kingdom') AND population < (SELECT population FROM world WHERE name = 'Germany');", conn)
print(result)

print("\n4")
result = pd.read_sql("SELECT w.name, ROUND(100.0 * w.population / g.population, 0), '%' AS percentage_of_germany FROM world w CROSS JOIN (SELECT population AS population FROM world WHERE name = 'Germany') g WHERE w.region = 'Europe' ORDER BY w.name;", conn)
print(result)

print("\n5")
result = pd.read_sql("SELECT region AS continent, name, area FROM (SELECT region, name, area, ROW_NUMBER() OVER (PARTITION BY region ORDER BY area DESC) AS rn FROM world) t WHERE rn = 1;", conn)
print(result)

print("\n6")
result = pd.read_sql("SELECT region FROM world GROUP BY region HAVING MAX(population) <= 25000000;", conn)
print(result)

1
            name  population
0     Bangladesh   147365352
1         Brazil   188078227
2          China  1313973713
3          India  1095351995
4      Indonesia   245452739
5       Pakistan   165803560
6  United States   298444215

2
              name  gdp_per_capita
0          Austria         30000.0
1          Belgium         29100.0
2          Denmark         31100.0
3          Finland         27400.0
4           France         27600.0
5          Germany         27600.0
6          Iceland         30900.0
7          Ireland         29600.0
8       Luxembourg         55100.0
9           Monaco         27000.0
10     Netherlands         28600.0
11          Norway         37800.0
12      San Marino         34600.0
13          Sweden         26800.0
14     Switzerland         32700.0
15  United Kingdom         27700.0

3
               name  population
0  Congo, Dem. Rep.    62660551
1             Egypt    78887007
2          Ethiopia    74777981
3            France    60876136
4    

## Job 6

In [54]:
print("1")
result = pd.read_sql("SELECT SUM(population) AS world_population FROM world;", conn)
print(result)

print("\n2")
result = pd.read_sql("SELECT region AS continent, SUM(population) AS total_population FROM world GROUP BY region ORDER BY total_population DESC;", conn)
print(result)

print("\n3")
result = pd.read_sql("SELECT region AS continent, SUM(gdp_per_capita) AS total_gdp FROM world GROUP BY region;", conn)
print(result)

result = pd.read_sql("SELECT region AS continent, SUM(gdp_per_capita * population) AS total_gdp_estimate FROM world GROUP BY region;", conn)
print(result)

print("\n4")
result = pd.read_sql("SELECT SUM(gdp_per_capita) AS africa_total_gdp FROM world WHERE region in ('SUB-SAHARAN AFRICA', 'NORTHERN AFRICA');", conn)
print(result)

print("\n5")
result = pd.read_sql("SELECT COUNT(*) AS big_countries_count FROM world WHERE area >= 1000000;", conn)
print(result)

print("\n6")
result = pd.read_sql("SELECT SUM(population) AS baltic_population FROM world WHERE name IN ('Estonia', 'Latvia', 'Lithuania');", conn)
print(result)

print("\n7")
result = pd.read_sql("SELECT region AS continent, COUNT(*) AS country_count FROM world GROUP BY region ORDER BY country_count DESC;", conn)
print(result)

print("\n8")
result = pd.read_sql("SELECT region AS continent, SUM(population) AS total_population FROM world GROUP BY region HAVING SUM(population) >= 100000000;", conn)
print(result)

1
   world_population
0        6524044551

2
               continent  total_population
0   ASIA (EX. NEAR EAST)        3687982236
1     SUB-SAHARAN AFRICA         749437000
2    LATIN AMER. & CARIB         561824599
3         WESTERN EUROPE         396339998
4       NORTHERN AMERICA         331672307
5    C.W. OF IND. STATES         280081548
6              NEAR EAST         195068377
7        NORTHERN AFRICA         161407133
8         EASTERN EUROPE         119914717
9                OCEANIA          33131662
10               BALTICS           7184974

3
               continent  total_gdp
0   ASIA (EX. NEAR EAST)   225500.0
1                BALTICS    33900.0
2    C.W. OF IND. STATES    48000.0
3         EASTERN EUROPE   117700.0
4    LATIN AMER. & CARIB   390700.0
5              NEAR EAST   167300.0
6        NORTHERN AFRICA    27300.0
7       NORTHERN AMERICA   130500.0
8                OCEANIA   173200.0
9     SUB-SAHARAN AFRICA   118500.0
10        WESTERN EUROPE   757300.0
    

## Job 7

In [55]:
print("0 - Create Tables")
conn.execute("CREATE TABLE IF NOT EXISTS game (id INTEGER, mdate DATE, stadium VARCHAR(100), team1 CHAR(3), team2 CHAR(3));")
conn.execute("CREATE TABLE IF NOT EXISTS eteam (id CHAR(3), teamname VARCHAR(50), coach VARCHAR(50));")
conn.execute("CREATE TABLE IF NOT EXISTS goal (matchid INTEGER, teamid CHAR(3), player VARCHAR(100), gtime INTEGER);")

conn.execute("INSERT INTO eteam VALUES ('POL','Poland','Franciszek Smuda');")
conn.execute("INSERT INTO eteam VALUES ('RUS','Russia','Dick Advocaat');")
conn.execute("INSERT INTO eteam VALUES ('CZE','Czech Republic','Michal Bílek');")
conn.execute("INSERT INTO eteam VALUES ('GRE','Greece','Fernando Santos');")
conn.execute("INSERT INTO eteam VALUES ('NED','Netherlands','Bert van Marwijk');")
conn.execute("INSERT INTO eteam VALUES ('DEN','Denmark','Morten Olsen');")
conn.execute("INSERT INTO eteam VALUES ('GER','Germany','Joachim Löw');")
conn.execute("INSERT INTO eteam VALUES ('POR','Portugal','Paulo Bento');")
conn.execute("INSERT INTO eteam VALUES ('ESP','Spain','Vicente del Bosque');")
conn.execute("INSERT INTO eteam VALUES ('ITA','Italy','Cesare Prandelli');")
conn.execute("INSERT INTO eteam VALUES ('IRL','Republic of Ireland','Giovanni Trapattoni');")
conn.execute("INSERT INTO eteam VALUES ('CRO','Croatia','Slaven Bilić');")
conn.execute("INSERT INTO eteam VALUES ('UKR','Ukraine','Oleh Blokhin');")
conn.execute("INSERT INTO eteam VALUES ('SWE','Sweden','Erik Hamrén');")
conn.execute("INSERT INTO eteam VALUES ('FRA','France','Laurent Blanc');")
conn.execute("INSERT INTO eteam VALUES ('ENG','England','Roy Hodgson');")

conn.execute("INSERT INTO game VALUES (1001,'8 June 2012','National Stadium, Warsaw','POL','GRE');")
conn.execute("INSERT INTO game VALUES (1002,'8 June 2012','Stadion Miejski (Wroclaw)','RUS','CZE');")
conn.execute("INSERT INTO game VALUES (1003,'12 June 2012','Stadion Miejski (Wroclaw)','GRE','CZE');")
conn.execute("INSERT INTO game VALUES (1004,'12 June 2012','National Stadium, Warsaw','POL','RUS');")
conn.execute("INSERT INTO game VALUES (1005,'16 June 2012','Stadion Miejski (Wroclaw)','CZE','POR');")
conn.execute("INSERT INTO game VALUES (1006,'16 June 2012','National Stadium, Warsaw','GRE','RUS');")
conn.execute("INSERT INTO game VALUES (1007,'9 June 2012','Metalist Stadium','NED','DEN');")
conn.execute("INSERT INTO game VALUES (1008,'9 June 2012','Arena Lviv','GER','POR');")
conn.execute("INSERT INTO game VALUES (1009,'13 June 2012','Arena Lviv','DEN','POR');")
conn.execute("INSERT INTO game VALUES (1010,'13 June 2012','Metalist Stadium','NED','GER');")
conn.execute("INSERT INTO game VALUES (1011,'17 June 2012','Metalist Stadium','POR','NED');")
conn.execute("INSERT INTO game VALUES (1012,'17 June 2012','Arena Lviv','DEN','GER');")
conn.execute("INSERT INTO game VALUES (1013,'10 June 2012','PGE Arena Gdansk','ESP','ITA');")
conn.execute("INSERT INTO game VALUES (1014,'10 June 2012','Stadion Miejski (Poznan)','IRL','CRO');")
conn.execute("INSERT INTO game VALUES (1015,'14 June 2012','Stadion Miejski (Poznan)','ITA','CRO');")
conn.execute("INSERT INTO game VALUES (1016,'14 June 2012','PGE Arena Gdansk','ESP','IRL');")
conn.execute("INSERT INTO game VALUES (1017,'18 June 2012','PGE Arena Gdansk','CRO','ESP');")
conn.execute("INSERT INTO game VALUES (1018,'18 June 2012','Stadion Miejski (Poznan)','ITA','IRL');")
conn.execute("INSERT INTO game VALUES (1019,'11 June 2012','Donbass Arena','FRA','ENG');")
conn.execute("INSERT INTO game VALUES (1020,'11 June 2012','Olimpiyskiy','UKR','SWE');")
conn.execute("INSERT INTO game VALUES (1021,'15 June 2012','Donbass Arena','UKR','FRA');")
conn.execute("INSERT INTO game VALUES (1022,'15 June 2012','Olimpiyskiy','SWE','ENG');")
conn.execute("INSERT INTO game VALUES (1023,'19 June 2012','Donbass Arena','ENG','UKR');")
conn.execute("INSERT INTO game VALUES (1024,'19 June 2012','Olimpiyskiy','SWE','FRA');")

conn.execute("INSERT INTO goal VALUES (1001,'POL','Robert Lewandowski',17);")
conn.execute("INSERT INTO goal VALUES (1001,'GRE','Dimitris Salpingidis',51);")
conn.execute("INSERT INTO goal VALUES (1002,'RUS','Alan Dzagoev',15);")
conn.execute("INSERT INTO goal VALUES (1002,'RUS','Roman Pavlyuchenko',82);")
conn.execute("INSERT INTO goal VALUES (1003,'GRE','Theofanis Gekas',53);")
conn.execute("INSERT INTO goal VALUES (1003,'CZE','Petr Jiráček',76);")
conn.execute("INSERT INTO goal VALUES (1004,'POL','Jakub Błaszczykowski',57);")
conn.execute("INSERT INTO goal VALUES (1005,'POR','Cristiano Ronaldo',79);")
conn.execute("INSERT INTO goal VALUES (1006,'RUS','Alan Dzagoev',24);")
conn.execute("INSERT INTO goal VALUES (1006,'RUS','Roman Shirokov',73);")
conn.execute("INSERT INTO goal VALUES (1006,'RUS','Andrey Arshavin',111);")
conn.execute("INSERT INTO goal VALUES (1007,'DEN','Michael Krohn-Dehli',24);")
conn.execute("INSERT INTO goal VALUES (1008,'POR','Cristiano Ronaldo',24);")
conn.execute("INSERT INTO goal VALUES (1008,'GER','Mario Gómez',72);")
conn.execute("INSERT INTO goal VALUES (1009,'DEN','Nicklas Bendtner',41);")
conn.execute("INSERT INTO goal VALUES (1009,'POR','Pepe',65);")
conn.execute("INSERT INTO goal VALUES (1010,'NED','Robin van Persie',73);")
conn.execute("INSERT INTO goal VALUES (1011,'POR','Cristiano Ronaldo',28);")
conn.execute("INSERT INTO goal VALUES (1011,'POR','Silvestre Varela',74);")
conn.execute("INSERT INTO goal VALUES (1012,'DEN','Michael Krohn-Dehli',24);")
conn.execute("INSERT INTO goal VALUES (1012,'GER','Lars Bender',80);")
conn.execute("INSERT INTO goal VALUES (1013,'ESP','Cesc Fàbregas',64);")
conn.execute("INSERT INTO goal VALUES (1013,'ESP','Alvaro Negredo',82);")
conn.execute("INSERT INTO goal VALUES (1013,'ESP','Alvaro Negredo',88);")
conn.execute("INSERT INTO goal VALUES (1014,'IRL','Sean St Ledger',19);")
conn.execute("INSERT INTO goal VALUES (1014,'CRO','Mario Mandžukić',32);")
conn.execute("INSERT INTO goal VALUES (1014,'CRO','Mario Mandžukić',48);")
conn.execute("INSERT INTO goal VALUES (1014,'CRO','Nikica Jelavić',75);")
conn.execute("INSERT INTO goal VALUES (1015,'ITA','Andrea Pirlo',39);")
conn.execute("INSERT INTO goal VALUES (1015,'CRO','Mario Mandžukić',72);")
conn.execute("INSERT INTO goal VALUES (1016,'ESP','Fernando Torres',4);")
conn.execute("INSERT INTO goal VALUES (1016,'ESP','David Silva',49);")
conn.execute("INSERT INTO goal VALUES (1016,'ESP','Fernando Torres',83);")
conn.execute("INSERT INTO goal VALUES (1016,'ESP','Juan Mata',87);")
conn.execute("INSERT INTO goal VALUES (1017,'ESP','Jesús Navas',88);")
conn.execute("INSERT INTO goal VALUES (1018,'ITA','Antonio Cassano',35);")
conn.execute("INSERT INTO goal VALUES (1018,'ITA','Mario Balotelli',90);")
conn.execute("INSERT INTO goal VALUES (1019,'FRA','Samir Nasri',39);")
conn.execute("INSERT INTO goal VALUES (1019,'ENG','Joleon Lescott',30);")
conn.execute("INSERT INTO goal VALUES (1020,'UKR','Andriy Shevchenko',55);")
conn.execute("INSERT INTO goal VALUES (1020,'UKR','Andriy Shevchenko',62);")
conn.execute("INSERT INTO goal VALUES (1020,'SWE','Zlatan Ibrahimović',52);")
conn.execute("INSERT INTO goal VALUES (1021,'UKR','Andriy Shevchenko',34);")
conn.execute("INSERT INTO goal VALUES (1021,'FRA','Yohan Cabaye',54);")
conn.execute("INSERT INTO goal VALUES (1022,'SWE','Glen Johnson',49);")
conn.execute("INSERT INTO goal VALUES (1022,'ENG','Andy Carroll',64);")
conn.execute("INSERT INTO goal VALUES (1023,'ENG','Wayne Rooney',48);")
conn.execute("INSERT INTO goal VALUES (1024,'FRA','Sébastien Squillaci',37);")

print("2")
result = pd.read_sql("SELECT g.matchid, g.player FROM goal g WHERE g.teamid = 'GER';", conn)
print(result)

print("\n3")
result = pd.read_sql("SELECT id, stadium, team1, team2 FROM game WHERE id = 1012;", conn)
print(result)

print("\n4")
result = pd.read_sql("SELECT go.player, go.teamid, ga.stadium, ga.mdate FROM goal go JOIN game ga ON ga.id = go.matchid WHERE go.teamid = 'GER';", conn)
print(result)

print("\n5")
result = pd.read_sql("SELECT ga.team1, ga.team2, go.player FROM goal go JOIN game ga ON ga.id = go.matchid WHERE go.player LIKE '%Mario%';", conn)
print(result)

print("\n6")
result = pd.read_sql("SELECT go.*, et.teamname, et.coach FROM goal go JOIN eteam et ON go.teamid = et.id;", conn)
print(result)

print("\n7")
result = pd.read_sql("SELECT go.player, go.teamid, et.coach, go.gtime FROM goal go JOIN eteam et ON go.teamid = et.id WHERE go.gtime <= 10;", conn)
print(result)

print("\n8")
result = pd.read_sql("SELECT ga.mdate, et.teamname FROM game ga JOIN eteam et ON ga.team1 = et.id WHERE et.coach = 'Fernando Santos';", conn)
print(result)

print("\n9")
result = pd.read_sql("SELECT go.player, go.matchid FROM goal go JOIN game ga ON go.matchid = ga.id WHERE ga.stadium = 'National Stadium, Warsaw';", conn)
print(result)

print("\n10")
result = pd.read_sql("SELECT teamid, COUNT(*) AS total_goals FROM goal GROUP BY teamid ORDER BY total_goals DESC;", conn)
print(result)

print("\n11")
result = pd.read_sql("SELECT ga.stadium, COUNT(*) AS goals_in_stadium FROM goal go JOIN game ga ON go.matchid = ga.id GROUP BY ga.stadium ORDER BY goals_in_stadium DESC;", conn)
print(result)

print("\n12")
result = pd.read_sql("SELECT ga.id AS match_id, ga.mdate AS match_date, COUNT(*) AS france_goals FROM game ga JOIN goal go ON go.matchid = ga.id WHERE go.teamid = 'FRA' GROUP BY ga.id, ga.mdate;", conn)
print(result)

0 - Create Tables
2
   matchid       player
0     1008  Mario Gómez
1     1012  Lars Bender

3
     id     stadium team1 team2
0  1012  Arena Lviv   DEN   GER

4
        player teamid     stadium         mdate
0  Mario Gómez    GER  Arena Lviv   9 June 2012
1  Lars Bender    GER  Arena Lviv  17 June 2012

5
  team1 team2           player
0   GER   POR      Mario Gómez
1   IRL   CRO  Mario Mandžukić
2   IRL   CRO  Mario Mandžukić
3   ITA   CRO  Mario Mandžukić
4   ITA   IRL  Mario Balotelli

6
    matchid teamid                player  gtime             teamname  \
0      1001    POL    Robert Lewandowski     17               Poland   
1      1001    GRE  Dimitris Salpingidis     51               Greece   
2      1002    RUS          Alan Dzagoev     15               Russia   
3      1002    RUS    Roman Pavlyuchenko     82               Russia   
4      1003    GRE       Theofanis Gekas     53               Greece   
5      1003    CZE          Petr Jiráček     76       Czech Republic  

## Job 8

In [56]:
# 1

# 2
conn.execute("CREATE TABLE IF NOT EXISTS Employees (employee_id INT PRIMARY KEY, first_name VARCHAR(100), last_name VARCHAR(100), birthdate DATE, position VARCHAR(100), department_id INT);")

# 3
conn.execute("CREATE TABLE IF NOT EXISTS Departments (department_id INT PRIMARY KEY, department_name VARCHAR(100), department_head INT, location VARCHAR(100));")

# 4
conn.execute("INSERT INTO Employees (employee_id, first_name, last_name, birthdate, position, department_id) VALUES (13, 'Liam', 'Martin', '1991-04-12', 'DevOps Engineer', 1), (14, 'Maya', 'Lopez', '1992-07-08', 'Data Engineer', 1), (15, 'Noah', 'Nguyen', '1990-11-02', 'Product Owner', 2), (16, 'Zara', 'Khan', '1994-02-21', 'UI Designer', 1), (17, 'Omar', 'Ben', '1993-09-15', 'QA Engineer', 1), (18, 'Clara', 'Duval', '1988-12-07', 'HR Specialist', 3);")

# 5
result = pd.read_sql("SELECT first_name, last_name, position FROM Employees;", conn)
print(result)

# 6
conn.execute("UPDATE Employees SET position = 'Senior DevOps Engineer' WHERE employee_id = 13;")

# 7
conn.execute("DELETE FROM Employees WHERE employee_id = 18;")

# 8
result = pd.read_sql("SELECT e.first_name, e.last_name, d.department_name, d.location FROM Employees e LEFT JOIN Departments d ON e.department_id = d.department_id;", conn)
print(result)

# 9
result = pd.read_sql("SELECT e.first_name, e.last_name, e.position FROM Employees e JOIN Departments d ON e.department_id = d.department_id WHERE d.department_name = 'IT';", conn)
print(result)

result = pd.read_sql("SELECT e.first_name, e.last_name, e.position FROM Employees e JOIN Departments d ON e.department_id = d.department_id WHERE d.department_name = 'Project Management';", conn)
print(result)

result = pd.read_sql("SELECT e.first_name, e.last_name, e.position FROM Employees e JOIN Departments d ON e.department_id = d.department_id WHERE d.department_name = 'Human Resources';", conn)
print(result)

# 10
result = pd.read_sql("SELECT d.department_name, d.location, (m.first_name || ' ' || m.last_name) AS manager_name FROM Departments d LEFT JOIN Employees m ON d.department_head = m.employee_id ORDER BY d.department_name;", conn)
print(result)

# 11
conn.execute("INSERT INTO Departments (department_id, department_name, department_head, location) VALUES (4, 'Marketing', 8, 'Branch Office West');")
conn.execute("UPDATE Employees SET department_id = 4 WHERE employee_id IN (8, 14);")

# 12
conn.execute("CREATE TABLE IF NOT EXISTS Project (project_id INT PRIMARY KEY, project_name VARCHAR(200), start_date DATE, end_date DATE, department_id INT);")
conn.execute("INSERT INTO Project (project_id, project_name, start_date, end_date, department_id) VALUES (1, 'Platform Migration', '2024-01-15', '2024-07-30', 1), (2, 'New Marketing Campaign', '2024-03-01', '2024-05-31', 4);")

  first_name last_name         position
0       Liam    Martin  DevOps Engineer
1       Maya     Lopez    Data Engineer
2       Noah    Nguyen    Product Owner
3       Zara      Khan      UI Designer
4       Omar       Ben      QA Engineer
5      Clara     Duval    HR Specialist
  first_name last_name department_name location
0       Liam    Martin            None     None
1       Maya     Lopez            None     None
2       Noah    Nguyen            None     None
3       Zara      Khan            None     None
4       Omar       Ben            None     None
Empty DataFrame
Columns: [first_name, last_name, position]
Index: []
Empty DataFrame
Columns: [first_name, last_name, position]
Index: []
Empty DataFrame
Columns: [first_name, last_name, position]
Index: []
Empty DataFrame
Columns: [department_name, location, manager_name]
Index: []


## Job 9

In [57]:
print("1")
result = pd.read_sql("SELECT name, literacy FROM world WHERE literacy IS NOT NULL ORDER BY literacy DESC LIMIT 10;", conn)
print(result)

print("\n2")
result = pd.read_sql("SELECT name, net_migration FROM world WHERE net_migration IS NOT NULL ORDER BY net_migration DESC LIMIT 15;", conn)
print(result)

print("\n3")
result = pd.read_sql("SELECT CASE WHEN birthrate < 10 THEN '<10' WHEN birthrate BETWEEN 10 AND 19.9 THEN '10-19.9' WHEN birthrate BETWEEN 20 AND 29.9 THEN '20-29.9' ELSE '>=30' END AS birthrate_bucket, AVG(infant_mortality) AS avg_infant_mortality, COUNT(*) as countries_in_bucket FROM world GROUP BY birthrate_bucket ORDER BY birthrate_bucket;", conn)
print(result)

print("\n4")
result = pd.read_sql("SELECT name, arable, crops FROM world WHERE arable IS NOT NULL ORDER BY arable DESC LIMIT 10;", conn)
print(result)

print("\n5")
result = pd.read_sql("SELECT name, birthrate, deathrate FROM world WHERE birthrate IS NOT NULL AND deathrate IS NOT NULL AND deathrate > birthrate;", conn)
print(result)

print("\n6")
result = pd.read_sql("SELECT name, infant_mortality, gdp_per_capita FROM world WHERE infant_mortality IS NOT NULL ORDER BY infant_mortality DESC LIMIT 20;", conn)
print(result)

1
             name  literacy
0         Andorra     100.0
1       Australia     100.0
2         Denmark     100.0
3         Finland     100.0
4   Liechtenstein     100.0
5      Luxembourg     100.0
6          Norway     100.0
7  Czech Republic      99.9
8         Iceland      99.9
9         Estonia      99.8

2
                  name  net_migration
0          Afghanistan          23.06
1       Cayman Islands          18.75
2                Qatar          16.29
3               Kuwait          14.18
4    Turks & Caicos Is          11.68
5            Singapore          11.53
6           San Marino          10.98
7             Anguilla          10.76
8   British Virgin Is.          10.01
9   N. Mariana Islands           9.61
10          Luxembourg           8.97
11              Monaco           7.75
12             Mayotte           6.78
13             Andorra           6.60
14              Jordan           6.59

3
  birthrate_bucket  avg_infant_mortality  countries_in_bucket
0          10-